In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [2]:
NBA2019_plays = pd.read_csv("nba_playbyplay_2018-2019.csv")

C:\Users\Can Sarioz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (72,90,91) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
list(NBA2019_plays.columns)

['Unnamed: 0',
 'EVENTMSGACTIONTYPE',
 'EVENTMSGTYPE',
 'EVENTNUM',
 'GAME_ID',
 'HOMEDESCRIPTION',
 'NEUTRALDESCRIPTION',
 'PCTIMESTRING',
 'PERIOD',
 'PERSON1TYPE',
 'PERSON2TYPE',
 'PERSON3TYPE',
 'PLAYER1_ID',
 'PLAYER1_NAME',
 'PLAYER1_TEAM_ABBREVIATION',
 'PLAYER1_TEAM_CITY',
 'PLAYER1_TEAM_ID',
 'PLAYER1_TEAM_NICKNAME',
 'PLAYER2_ID',
 'PLAYER2_NAME',
 'PLAYER2_TEAM_ABBREVIATION',
 'PLAYER2_TEAM_CITY',
 'PLAYER2_TEAM_ID',
 'PLAYER2_TEAM_NICKNAME',
 'PLAYER3_ID',
 'PLAYER3_NAME',
 'PLAYER3_TEAM_ABBREVIATION',
 'PLAYER3_TEAM_CITY',
 'PLAYER3_TEAM_ID',
 'PLAYER3_TEAM_NICKNAME',
 'SCORE',
 'SCOREMARGIN',
 'VISITORDESCRIPTION',
 'WCTIMESTRING',
 'HOME_TEAM',
 'AWAY_TEAM',
 'HOME_SCORE',
 'AWAY_SCORE',
 'DIFFERENCE',
 'TIME',
 'TEAM',
 'TYPE',
 'SUB_TYPE',
 'ASSIST_PLAYER_ID',
 'ASSIST_COUNT',
 'BLOCK_PLAYER_ID',
 'BLOCK_COUNT',
 'REBOUND_PLAYER_ID',
 'REBOUND_TEAM',
 'REBOUND_OFFENSIVE_COUNT',
 'REBOUND_DEFENSIVE_COUNT',
 'JUMP_BALL_HOME_PLAYER_ID',
 'JUMP_BALL_AWAY_PLAYER_ID',
 'JUM

## Creating the Added Columns (with sample data to test)

The needed column additions are made in the excel for the 2018-19 NBA play-by-play data.

In [4]:
NBA2019_plays['DIFFERENCE'] = abs(NBA2019_plays['HOME_SCORE'] - NBA2019_plays['AWAY_SCORE'])

In [5]:
short_nba = NBA2019_plays.sample(50)
short_nba.head()

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,AWAY_PLAYER_ID_1_PLAY_TIME,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME,3PT_ATT_HOME,3PT_ATT_AWAY,3PT_ATT,3PT_MADE_PRE,3PT_MADE
38328,314,0,4,461,21801037,Kleber REBOUND (Off:4 Def:5),NaN,11:14,4,4.0,...,586.0,760.0,853.0,1459.0,1016.0,False,False,False,False,NaN
284972,3,0,4,8,21800330,NaN,NaN,11:40,1,5.0,...,20.0,20.0,20.0,20.0,20.0,False,False,False,False,NaN
149291,418,0,4,586,21800045,Ingles REBOUND (Off:2 Def:5),NaN,4:59,4,4.0,...,1012.0,1165.0,1781.0,1852.0,1933.0,False,False,False,False,NaN
71379,406,1,5,591,21801110,McCaw STEAL (3 STL),NaN,2:24,4,5.0,...,1168.0,1237.0,1543.0,1860.0,2260.0,False,False,False,False,NaN
447836,4,0,4,10,21800676,Gallinari REBOUND (Off:0 Def:1),NaN,11:13,1,4.0,...,47.0,47.0,47.0,47.0,47.0,False,False,False,False,NaN


In [6]:
short_nba['3PT_ATT_HOME']= short_nba['HOMEDESCRIPTION'].str.contains('3PT')
short_nba['3PT_ATT_AWAY']= short_nba['VISITORDESCRIPTION'].str.contains('3PT')

In [7]:
short_nba['3PT_ATT'] = (short_nba['3PT_ATT_HOME'] | short_nba['3PT_ATT_AWAY'])
short_nba['3PT_MADE'] = (short_nba['3PT_ATT'] & short_nba['SHOT_MADE'])

In [8]:
short_nba = short_nba[['GAME_ID', 'PCTIMESTRING', 'PERIOD', 'DIFFERENCE','HOMEDESCRIPTION', 'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION', 'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ABBREVIATION', 'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ABBREVIATION', 'HOME_TEAM', 'AWAY_TEAM', 'FREE_THROW_PLAYER_ID', 'FREE_THROW_MADE', 'SHOT_PLAYER_ID', 'SHOT_MADE', '3PT_MADE']]

In [9]:
short_nba

,GAME_ID,PCTIMESTRING,PERIOD,DIFFERENCE,HOMEDESCRIPTION,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER2_ID,PLAYER2_NAME,...,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,HOME_TEAM,AWAY_TEAM,FREE_THROW_PLAYER_ID,FREE_THROW_MADE,SHOT_PLAYER_ID,SHOT_MADE,3PT_MADE
38328,21801037,11:14,4,12,Kleber REBOUND (Off:4 Def:5),1628467,Maxi Kleber,DAL,0,NaN,...,0,NaN,NaN,Cavaliers,Mavericks,NaN,NaN,NaN,NaN,False
284972,21800330,11:40,1,0,NaN,201572,Brook Lopez,MIL,0,NaN,...,0,NaN,NaN,Bucks,Knicks,NaN,NaN,NaN,NaN,False
149291,21800045,4:59,4,10,Ingles REBOUND (Off:2 Def:5),204060,Joe Ingles,UTA,0,NaN,...,0,NaN,NaN,Grizzlies,Jazz,NaN,NaN,NaN,NaN,False
71379,21801110,2:24,4,15,McCaw STEAL (3 STL),1627782,Wayne Selden,CHI,1627775,Patrick McCaw,...,0,NaN,NaN,Bulls,Raptors,NaN,NaN,NaN,NaN,False
447836,21800676,11:13,1,2,Gallinari REBOUND (Off:0 Def:1),201568,Danilo Gallinari,LAC,0,NaN,...,0,NaN,NaN,Warriors,Clippers,NaN,NaN,NaN,NaN,False
345510,21800459,8:44,4,11,NaN,201936,Tyreke Evans,IND,203506,Victor Oladipo,...,0,NaN,NaN,Pacers,Raptors,NaN,NaN,NaN,NaN,False
492724,21800771,7:27,4,18,NaN,202694,Marcus Morris,BOS,1626179,Terry Rozier,...,0,NaN,NaN,Celtics,Knicks,NaN,NaN,202694.0,True,False
200800,21800152,7:29,4,8,TRAIL BLAZERS Timeout: Regular (Full 4 Short 0),1610612757,NaN,NaN,0,NaN,...,0,NaN,NaN,Bucks,Trail Blazers,NaN,NaN,NaN,NaN,False
524198,21800839,0:02,1,6,MISS Clarkson 25' 3PT Step Back Jump Shot,203903,Jordan Clarkson,CLE,0,NaN,...,0,NaN,NaN,Cavaliers,Knicks,NaN,NaN,203903.0,False,False
404679,21800584,3:28,3,8,NaN,1610612751,NaN,NaN,0,NaN,...,0,NaN,NaN,Nets,Bulls,NaN,NaN,NaN,NaN,False


## Filtering the Data

In [10]:
nba2019_plays = NBA2019_plays[['GAME_ID', 'PCTIMESTRING', 'PERIOD', 'DIFFERENCE','HOMEDESCRIPTION', 'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION', 'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ABBREVIATION', 'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ABBREVIATION', 'HOME_TEAM', 'AWAY_TEAM', 'FREE_THROW_PLAYER_ID', 'FREE_THROW_MADE', 'SHOT_PLAYER_ID', 'SHOT_MADE', '3PT_MADE']]

In [11]:
nba2019_plays.head()

,GAME_ID,PCTIMESTRING,PERIOD,DIFFERENCE,HOMEDESCRIPTION,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER2_ID,PLAYER2_NAME,...,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,HOME_TEAM,AWAY_TEAM,FREE_THROW_PLAYER_ID,FREE_THROW_MADE,SHOT_PLAYER_ID,SHOT_MADE,3PT_MADE
0,21801052,12:00,1,0,NaN,0,NaN,NaN,0,NaN,...,0,NaN,NaN,Celtics,Nuggets,NaN,NaN,NaN,NaN,NaN
1,21801052,12:00,1,0,Jump Ball Horford vs. Jokic: Tip to Millsap,201143,Al Horford,BOS,203999,Nikola Jokic,...,200794,Paul Millsap,DEN,Celtics,Nuggets,NaN,NaN,NaN,NaN,NaN
2,21801052,11:41,1,0,NaN,200794,Paul Millsap,DEN,0,NaN,...,0,NaN,NaN,Celtics,Nuggets,NaN,NaN,200794.0,False,False
3,21801052,11:39,1,0,Tatum REBOUND (Off:0 Def:1),1628369,Jayson Tatum,BOS,0,NaN,...,0,NaN,NaN,Celtics,Nuggets,NaN,NaN,NaN,NaN,NaN
4,21801052,11:34,1,0,MISS Horford 15' Jump Shot,201143,Al Horford,BOS,0,NaN,...,0,NaN,NaN,Celtics,Nuggets,NaN,NaN,201143.0,False,NaN


#### Filtering last quarter and overtime(s)

In [12]:
nba2019_plays = nba2019_plays.loc[nba2019_plays['PERIOD'] >= 4, :]

#### Couldn't figure out how to filter with time

In [13]:
nba2019_plays['MINUTE'] = [100*int(i[0]) + int(i[1]) for i in nba2019_plays['PCTIMESTRING'].str.split(':')]

In [14]:
nba2019_plays = nba2019_plays.loc[nba2019_plays['MINUTE'] < 500]

#### Filtering the difference

In [15]:
nba2019_plays = nba2019_plays.loc[abs(nba2019_plays['DIFFERENCE']) <= 6, :]

### 1 Player Test

In [16]:
jaylen_brown_19plays = nba2019_plays.loc[nba2019_plays['PLAYER1_ID'] == 1627759]
jaylen_brown_19plays.shape

(55, 22)

In [17]:
jaylen_brown_ft = jaylen_brown_19plays['FREE_THROW_MADE'].sum() / jaylen_brown_19plays['FREE_THROW_MADE'].count()

In [18]:
jaylen_brown_fg = jaylen_brown_19plays['SHOT_MADE'].sum() / jaylen_brown_19plays['SHOT_MADE'].count()

In [19]:
jaylen_brown_3pt = jaylen_brown_19plays['3PT_MADE'].sum() / jaylen_brown_19plays['3PT_MADE'].count()

In [20]:
jaylen_brown_fg, jaylen_brown_3pt, jaylen_brown_ft

(0.5333333333333333, 0.42857142857142855, 0.3333333333333333)

In [21]:
jaylen_brown_19plays['SHOT_MADE'].count(),jaylen_brown_19plays['3PT_MADE'].count(), jaylen_brown_19plays['FREE_THROW_MADE'].count()

(15, 7, 3)

In [22]:
jaylen_brown_total_plays = NBA2019_plays.loc[NBA2019_plays['PLAYER1_ID'] == 1627759]
jaylen_brown_ft_total = jaylen_brown_total_plays['FREE_THROW_MADE'].sum() / jaylen_brown_total_plays['FREE_THROW_MADE'].count()
jaylen_brown_fg_total = jaylen_brown_total_plays['SHOT_MADE'].sum() / jaylen_brown_total_plays['SHOT_MADE'].count()
jaylen_brown_3pt_total = jaylen_brown_total_plays['3PT_MADE'].sum() / jaylen_brown_total_plays['3PT_MADE'].count()
jaylen_brown_fg_total, jaylen_brown_total_plays['SHOT_MADE'].count(), jaylen_brown_3pt_total, jaylen_brown_total_plays['3PT_MADE'].count(), jaylen_brown_ft_total, jaylen_brown_total_plays['FREE_THROW_MADE'].count()

(0.46464646464646464, 792, 0.3442028985507246, 276, 0.6584158415841584, 202)

### Functions

In [23]:
nba2019_plays.head(10)

,GAME_ID,PCTIMESTRING,PERIOD,DIFFERENCE,HOMEDESCRIPTION,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER2_ID,PLAYER2_NAME,...,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,HOME_TEAM,AWAY_TEAM,FREE_THROW_PLAYER_ID,FREE_THROW_MADE,SHOT_PLAYER_ID,SHOT_MADE,3PT_MADE,MINUTE
444,21801052,1:22,4,5,Irving 3PT Turnaround Jump Shot (30 PTS),202681,Kyrie Irving,BOS,0,NaN,...,NaN,NaN,Celtics,Nuggets,NaN,NaN,202681.0,True,True,122
843,21801069,4:55,4,2,Dunn 8' Turnaround Hook Shot (18 PTS),1627739,Kris Dunn,CHI,0,NaN,...,NaN,NaN,Bulls,Wizards,NaN,NaN,1627739.0,True,NaN,455
844,21801069,4:37,4,2,NaN,203078,Bradley Beal,WAS,0,NaN,...,NaN,NaN,Bulls,Wizards,NaN,NaN,203078.0,False,False,437
845,21801069,4:33,4,2,NaN,203953,Jabari Parker,WAS,0,NaN,...,NaN,NaN,Bulls,Wizards,NaN,NaN,NaN,NaN,NaN,433
846,21801069,4:28,4,2,NaN,1628972,Troy Brown Jr.,WAS,0,NaN,...,NaN,NaN,Bulls,Wizards,NaN,NaN,1628972.0,False,False,428
847,21801069,4:25,4,2,Dunn REBOUND (Off:0 Def:5),1627739,Kris Dunn,CHI,0,NaN,...,NaN,NaN,Bulls,Wizards,NaN,NaN,NaN,NaN,NaN,425
848,21801069,4:15,4,2,MISS Lopez 3' Driving Hook Shot,201577,Robin Lopez,CHI,0,NaN,...,Thomas Bryant,WAS,Bulls,Wizards,NaN,NaN,201577.0,False,NaN,415
849,21801069,4:12,4,2,NaN,1628418,Thomas Bryant,WAS,0,NaN,...,NaN,NaN,Bulls,Wizards,NaN,NaN,NaN,NaN,NaN,412
850,21801069,4:07,4,2,Luwawu-Cabarrot STEAL (2 STL),203078,Bradley Beal,WAS,1627789,Timothe Luwawu-Cabarrot,...,NaN,NaN,Bulls,Wizards,NaN,NaN,NaN,NaN,NaN,407
851,21801069,4:01,4,5,Harrison 24' 3PT Jump Shot (15 PTS) (Dunn 11 AST),1627885,Shaquille Harrison,CHI,1627739,Kris Dunn,...,NaN,NaN,Bulls,Wizards,NaN,NaN,1627885.0,True,True,401


In [24]:
def name_getter(data, id_number):
    return list(data.loc[data['PLAYER1_ID'] == id_number]['PLAYER1_NAME'].value_counts().index)

In [25]:
def fg_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
def fg_made_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['SHOT_MADE'].sum()
def fg_attempt_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['SHOT_MADE'].count()

In [26]:
def ft_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
def ft_made_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['FREE_THROW_MADE'].sum()
def ft_attempt_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['FREE_THROW_MADE'].count()

In [27]:
def three_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return sum(player_filtered.loc[player_filtered['3PT_MADE'] == True]['3PT_MADE']) / sum(player_filtered.loc[player_filtered['3PT_ATT'] == True]['3PT_ATT'])
def three_made_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return sum(player_filtered.loc[player_filtered['3PT_MADE'] == True]['3PT_MADE'])
def three_attempt_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return sum(player_filtered.loc[player_filtered['3PT_ATT'] == True]['3PT_ATT'])

In [28]:
def year_getter(file):
    pattern = r'.*\-(\d+).*'
    return re.findall(pattern, "nba_playbyplay_2018-2019.csv")

### Building the Player Dataframe

In [29]:
counts2019 = nba2019_plays.groupby('PLAYER1_NAME').size().to_frame().rename(columns = {0: 'Count'})

In [30]:
sum(counts2019.loc[counts2019['Count']>55]['Count']) / sum(counts2019['Count'])

0.783242746689122

In [31]:
counts2019.loc[counts2019['Count']>55].shape

(174, 1)

In [32]:
players2019 = nba2019_plays.groupby('PLAYER1_ID')[['PLAYER1_NAME']].agg(pd.unique)

In [33]:
players2019['FG%'] = [fg_getter(nba2019_plays, x) for x in players2019.index]
players2019['FG MADE'] = [fg_made_getter(nba2019_plays, x) for x in players2019.index]
players2019['FG ATTEMPTED'] = [fg_attempt_getter(nba2019_plays, x) for x in players2019.index]
players2019['FT%'] = [ft_getter(nba2019_plays, x) for x in players2019.index]
players2019['FT MADE'] = [ft_made_getter(nba2019_plays, x) for x in players2019.index]
players2019['FT ATTEMPTED'] = [ft_attempt_getter(nba2019_plays, x) for x in players2019.index]
players2019['3PT%'] = [three_getter(nba2019_plays, x) for x in players2019.index]
players2019['3PT MADE'] = [three_made_getter(nba2019_plays, x) for x in players2019.index]
players2019['3PT ATTEMPTED'] = [three_attempt_getter(nba2019_plays, x) for x in players2019.index]


<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

KeyError: '3PT_ATT'

In [ ]:
players2019['TOTAL FG%'] = [fg_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL FG MADE'] = [fg_made_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL FG ATTEMPTED'] = [fg_attempt_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL FT%'] = [ft_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL FT MADE'] = [ft_made_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL FT ATTEMPTED'] = [ft_attempt_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL 3PT%'] = [three_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL 3PT MADE'] = [three_made_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL 3PT ATTEMPTED'] = [three_attempt_getter(NBA2019_plays, x) for x in players2019.index]


In [ ]:
players2019 = players2019.loc[players2019['PLAYER1_NAME'].isin(list(counts2019.loc[counts2019['Count']>127].index))]

In [ ]:
players2019['FG DIFFERENCE'] = players2019['FG%'] - players2019['TOTAL FG%']
players2019['FT DIFFERENCE'] = players2019['FT%'] - players2019['TOTAL FT%']
players2019['3PT DIFFERENCE'] = players2019['3PT%'] - players2019['TOTAL 3PT%']

In [ ]:
players2019['YEAR'] = [year_getter("nba_playbyplay_2018-2019.csv")[0] for x in players2019.index]

In [ ]:
players2019 = players2019[['PLAYER1_NAME', 'FG MADE', 'FG ATTEMPTED', 'FG%', 'TOTAL FG MADE', 'TOTAL FG ATTEMPTED', 'TOTAL FG%', 'FG DIFFERENCE','FT MADE', 'FT ATTEMPTED', 'FT%','TOTAL FT MADE', 'TOTAL FT ATTEMPTED', 'TOTAL FT%', 'FT DIFFERENCE', '3PT MADE', '3PT ATTEMPTED', '3PT%', 'TOTAL 3PT MADE', 'TOTAL 3PT ATTEMPTED', 'TOTAL 3PT%', '3PT DIFFERENCE', 'YEAR']]

In [ ]:
players2019.head()

In [ ]:
players2019[['FG DIFFERENCE', 'FT DIFFERENCE','3PT DIFFERENCE']].sum(axis=0, skipna=True)

In [ ]:
players2019.shape, players2019.loc[players2019['FG DIFFERENCE'] >= 0].shape, players2019.loc[players2019['FT DIFFERENCE'] >= 0].shape, players2019.loc[players2019['3PT DIFFERENCE'] >= 0].shape

### Builder Function

In [90]:
def builder(file, time, difference, min_plays):
    full_data = pd.read_csv(file)
    data = full_data.copy()
    
    full_data['DIFFERENCE'] = abs(full_data['HOME_SCORE'] - full_data['AWAY_SCORE'])
    full_data['3PT_ATT_HOME']= pd.array(full_data['HOMEDESCRIPTION'].str.contains('3PT'), dtype="boolean")
    full_data['3PT_ATT_AWAY']= pd.array(full_data['VISITORDESCRIPTION'].str.contains('3PT'), dtype="boolean")
    full_data['3PT_ATT'] = ((full_data['3PT_ATT_HOME']) | (full_data['3PT_ATT_AWAY']))
    full_data['3PT_MADE'] = ((full_data['3PT_ATT']) & (full_data['SHOT_MADE']))
    
    data['DIFFERENCE'] = abs(data['HOME_SCORE'] - data['AWAY_SCORE'])
    data['3PT_ATT_HOME']= pd.array(data['HOMEDESCRIPTION'].str.contains('3PT'), dtype="boolean")
    data['3PT_ATT_AWAY']= pd.array(data['VISITORDESCRIPTION'].str.contains('3PT'), dtype="boolean")
    data['3PT_ATT'] = ((data['3PT_ATT_HOME']) | (data['3PT_ATT_AWAY']))
    data['3PT_MADE'] = ((data['3PT_ATT']) & (data['SHOT_MADE']))
    data = data[['GAME_ID', 'PCTIMESTRING', 'PERIOD', 'DIFFERENCE','HOMEDESCRIPTION', 'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION', 'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ABBREVIATION', 'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ABBREVIATION', 'HOME_TEAM', 'AWAY_TEAM', 'FREE_THROW_PLAYER_ID', 'FREE_THROW_MADE', 'SHOT_PLAYER_ID', 'SHOT_MADE', '3PT_MADE']]
    data = data.loc[data['PERIOD'] >= 4, :]
    data['MINUTE'] = [100*int(i[0]) + int(i[1]) for i in data['PCTIMESTRING'].str.split(':')]
    data = data.loc[data['MINUTE'] < time]
    data = data.loc[abs(data['DIFFERENCE']) <= difference, :]
    
    
    counts = data.groupby('PLAYER1_NAME').size().to_frame().rename(columns = {0: 'Count'})
    players = data.groupby(['PLAYER1_ID','PLAYER1_NAME'])['PLAYER1_NAME'].agg(pd.unique).to_frame().droplevel(1)
    players['FG%'] = [fg_getter(data, x) for x in players.index]
    players['FG MADE'] = [fg_made_getter(data, x) for x in players.index]
    players['FG ATTEMPTED'] = [fg_attempt_getter(data, x) for x in players.index]
    players['FT%'] = [ft_getter(data, x) for x in players.index]
    players['FT MADE'] = [ft_made_getter(data, x) for x in players.index]
    players['FT ATTEMPTED'] = [ft_attempt_getter(data, x) for x in players.index]
    players['3PT%'] = [three_getter(data, x) for x in players.index]
    players['3PT MADE'] = [three_made_getter(data, x) for x in players.index]
    players['3PT ATTEMPTED'] = [three_attempt_getter(data, x) for x in players.index]
    
    players['TOTAL FG%'] = [fg_getter(full_data, x) for x in players.index]
    players['TOTAL FG MADE'] = [fg_made_getter(full_data, x) for x in players.index]
    players['TOTAL FG ATTEMPTED'] = [fg_attempt_getter(full_data, x) for x in players.index]
    players['TOTAL FT%'] = [ft_getter(full_data, x) for x in players.index]
    players['TOTAL FT MADE'] = [ft_made_getter(full_data, x) for x in players.index]
    players['TOTAL FT ATTEMPTED'] = [ft_attempt_getter(full_data, x) for x in players.index]
    players['TOTAL 3PT%'] = [three_getter(full_data, x) for x in players.index]
    players['TOTAL 3PT MADE'] = [three_made_getter(full_data, x) for x in players.index]
    players['TOTAL 3PT ATTEMPTED'] = [three_attempt_getter(full_data, x) for x in players.index]
    
    players = players.loc[players['PLAYER1_NAME'].isin(list(counts.loc[counts['Count']>min_plays].index))]
    
    players['FG DIFFERENCE'] = players['FG%'] - players['TOTAL FG%']
    players['FT DIFFERENCE'] = players['FT%'] - players['TOTAL FT%']
    players['3PT DIFFERENCE'] = players['3PT%'] - players['TOTAL 3PT%']
    
    players['YEAR'] = [year_getter(file)[0] for x in players.index]
    
    players = players[['PLAYER1_NAME', 'FG MADE', 'FG ATTEMPTED', 'FG%', 'TOTAL FG MADE', 'TOTAL FG ATTEMPTED', 'TOTAL FG%', 'FG DIFFERENCE','FT MADE', 'FT ATTEMPTED', 'FT%','TOTAL FT MADE', 'TOTAL FT ATTEMPTED', 'TOTAL FT%', 'FT DIFFERENCE', '3PT MADE', '3PT ATTEMPTED', '3PT%', 'TOTAL 3PT MADE', 'TOTAL 3PT ATTEMPTED', 'TOTAL 3PT%', '3PT DIFFERENCE', 'YEAR']]
    
    return players

In [ ]:
test_players2019 = builder("test_2018-2019.csv", 500, 6, 127)

In [39]:
test_players2019.shape, players2019.shape

NameError: name 'test_players2019' is not defined

In [137]:
players2019.equals(test_players2019)

False

In [138]:
test_players2019.head()

,PLAYER1_NAME,FG MADE,FG ATTEMPTED,FG%,TOTAL FG MADE,TOTAL FG ATTEMPTED,TOTAL FG%,FG DIFFERENCE,FT MADE,FT ATTEMPTED,...,TOTAL FT%,FT DIFFERENCE,3PT MADE,3PT ATTEMPTED,3PT%,TOTAL 3PT MADE,TOTAL 3PT ATTEMPTED,TOTAL 3PT%,3PT DIFFERENCE,YEAR
PLAYER1_ID,,,,,,,,,,,,,,,,,,,,,
2544,LeBron James,37,86,0.430233,558,1095,0.509589,-0.079356,39,55,...,0.665072,0.044019,12,62,0.193548,111,762,0.145669,0.047879,2019
2548,Dwyane Wade,32,88,0.363636,416,960,0.433333,-0.069697,10,14,...,0.708155,0.006131,7,68,0.102941,86,741,0.116059,-0.013118,2019
101150,Lou Williams,41,92,0.445652,484,1138,0.425308,0.020345,33,39,...,0.876289,-0.030135,10,67,0.149254,105,898,0.116927,0.032327,2019
200746,LaMarcus Aldridge,31,60,0.516667,684,1319,0.518575,-0.001908,36,44,...,0.847087,-0.028906,0,31,0.000000,10,705,0.014184,-0.014184,2019
200755,JJ Redick,24,65,0.369231,452,1027,0.440117,-0.070886,17,21,...,0.894118,-0.084594,13,55,0.236364,240,865,0.277457,-0.041093,2019


In [139]:
players2019.head()

,PLAYER1_NAME,FG MADE,FG ATTEMPTED,FG%,TOTAL FG MADE,TOTAL FG ATTEMPTED,TOTAL FG%,FG DIFFERENCE,FT MADE,FT ATTEMPTED,...,TOTAL FT%,FT DIFFERENCE,3PT MADE,3PT ATTEMPTED,3PT%,TOTAL 3PT MADE,TOTAL 3PT ATTEMPTED,TOTAL 3PT%,3PT DIFFERENCE,YEAR
PLAYER1_ID,,,,,,,,,,,,,,,,,,,,,
2544,LeBron James,37,86,0.430233,558,1095,0.509589,-0.079356,39,55,...,0.665072,0.044019,12,38,0.315789,111,327,0.339450,-0.023660,2019
2548,Dwyane Wade,32,88,0.363636,416,960,0.433333,-0.069697,10,14,...,0.708155,0.006131,7,32,0.218750,86,261,0.329502,-0.110752,2019
101150,Lou Williams,41,92,0.445652,484,1138,0.425308,0.020345,33,39,...,0.876289,-0.030135,10,29,0.344828,105,291,0.360825,-0.015997,2019
200746,LaMarcus Aldridge,31,60,0.516667,684,1319,0.518575,-0.001908,36,44,...,0.847087,-0.028906,0,1,0.000000,10,42,0.238095,-0.238095,2019
200755,JJ Redick,24,65,0.369231,452,1027,0.440117,-0.070886,17,21,...,0.894118,-0.084594,13,42,0.309524,240,605,0.396694,-0.087170,2019


In [59]:
NBA2019_plays.head()

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,AWAY_PLAYER_ID_1_PLAY_TIME,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME,3PT_ATT_HOME,3PT_ATT_AWAY,3PT_ATT,3PT_MADE_PRE,3PT_MADE
0,0,0,12,2,21801052,NaN,NaN,12:00,1,0.0,...,NaN,NaN,NaN,NaN,NaN,False,False,False,False,NaN
1,1,0,10,4,21801052,Jump Ball Horford vs. Jokic: Tip to Millsap,NaN,12:00,1,4.0,...,0.0,0.0,0.0,0.0,0.0,False,False,False,False,NaN
2,2,1,2,7,21801052,NaN,NaN,11:41,1,5.0,...,19.0,19.0,19.0,19.0,19.0,False,True,True,False,False
3,3,0,4,8,21801052,Tatum REBOUND (Off:0 Def:1),NaN,11:39,1,4.0,...,21.0,21.0,21.0,21.0,21.0,False,False,False,False,NaN
4,4,1,2,9,21801052,MISS Horford 15' Jump Shot,NaN,11:34,1,4.0,...,26.0,26.0,26.0,26.0,26.0,False,False,False,False,NaN


In [60]:
test = pd.read_csv('test_2018-2019.csv')

C:\Users\Can Sarioz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (71,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [93]:
test['3PT_ATT_HOME']= pd.array(test['HOMEDESCRIPTION'].str.contains('3PT'), dtype="boolean")
test['3PT_ATT_AWAY']= pd.array(test['VISITORDESCRIPTION'].str.contains('3PT'), dtype="boolean")
test['3PT_ATT'] = ((test['3PT_ATT_HOME']) | (test['3PT_ATT_AWAY']))
test['3PT_MADE'] = ((test['3PT_ATT']) & (test['SHOT_MADE']))
test.head()

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,HOME_PLAYER_ID_5_PLAY_TIME,AWAY_PLAYER_ID_1_PLAY_TIME,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME,3PT_ATT_HOME,3PT_ATT_AWAY,3PT_ATT,3PT_MADE
0,0,0,12,2,21801052,NaN,NaN,12:00,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1,1,0,10,4,21801052,Jump Ball Horford vs. Jokic: Tip to Millsap,NaN,12:00,1,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,<NA>,<NA>,<NA>
2,2,1,2,7,21801052,NaN,NaN,11:41,1,5.0,...,19.0,19.0,19.0,19.0,19.0,19.0,<NA>,True,True,False
3,3,0,4,8,21801052,Tatum REBOUND (Off:0 Def:1),NaN,11:39,1,4.0,...,21.0,21.0,21.0,21.0,21.0,21.0,False,<NA>,<NA>,<NA>
4,4,1,2,9,21801052,MISS Horford 15' Jump Shot,NaN,11:34,1,4.0,...,26.0,26.0,26.0,26.0,26.0,26.0,False,<NA>,<NA>,False


In [103]:
test_lebron = test.loc[test['PLAYER1_NAME'] == 'LeBron James']
test_lebron_3pt = sum(test_lebron.loc[test_lebron['3PT_MADE'] == True]['3PT_MADE']) / sum(test_lebron.loc[test_lebron['3PT_ATT'] == True]['3PT_ATT'])
test_lebron_3pt

0.3394495412844037

In [118]:
three_getter(test, 2544)

0.3394495412844037

In [126]:
full_data = pd.read_csv('test_2018-2019.csv')

    
full_data['DIFFERENCE'] = abs(full_data['HOME_SCORE'] - full_data['AWAY_SCORE'])
full_data['3PT_ATT_HOME']= pd.array(full_data['HOMEDESCRIPTION'].str.contains('3PT'), dtype="boolean")
full_data['3PT_ATT_AWAY']= pd.array(full_data['VISITORDESCRIPTION'].str.contains('3PT'), dtype="boolean")
full_data['3PT_ATT'] = ((full_data['3PT_ATT_HOME']) | (full_data['3PT_ATT_AWAY']))
full_data['3PT_MADE'] = ((full_data['3PT_ATT']) & (full_data['SHOT_MADE']))
full_data.head()

C:\Users\Can Sarioz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (71,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,AWAY_PLAYER_ID_1_PLAY_TIME,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME,DIFFERENCE,3PT_ATT_HOME,3PT_ATT_AWAY,3PT_ATT,3PT_MADE
0,0,0,12,2,21801052,NaN,NaN,12:00,1,0.0,...,NaN,NaN,NaN,NaN,NaN,0,<NA>,<NA>,<NA>,<NA>
1,1,0,10,4,21801052,Jump Ball Horford vs. Jokic: Tip to Millsap,NaN,12:00,1,4.0,...,0.0,0.0,0.0,0.0,0.0,0,False,<NA>,<NA>,<NA>
2,2,1,2,7,21801052,NaN,NaN,11:41,1,5.0,...,19.0,19.0,19.0,19.0,19.0,0,<NA>,True,True,False
3,3,0,4,8,21801052,Tatum REBOUND (Off:0 Def:1),NaN,11:39,1,4.0,...,21.0,21.0,21.0,21.0,21.0,0,False,<NA>,<NA>,<NA>
4,4,1,2,9,21801052,MISS Horford 15' Jump Shot,NaN,11:34,1,4.0,...,26.0,26.0,26.0,26.0,26.0,0,False,<NA>,<NA>,False


In [127]:
NBA2019_plays.head()

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,AWAY_PLAYER_ID_1_PLAY_TIME,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME,3PT_ATT_HOME,3PT_ATT_AWAY,3PT_ATT,3PT_MADE_PRE,3PT_MADE
0,0,0,12,2,21801052,NaN,NaN,12:00,1,0.0,...,NaN,NaN,NaN,NaN,NaN,False,False,False,False,NaN
1,1,0,10,4,21801052,Jump Ball Horford vs. Jokic: Tip to Millsap,NaN,12:00,1,4.0,...,0.0,0.0,0.0,0.0,0.0,False,False,False,False,NaN
2,2,1,2,7,21801052,NaN,NaN,11:41,1,5.0,...,19.0,19.0,19.0,19.0,19.0,False,True,True,False,False
3,3,0,4,8,21801052,Tatum REBOUND (Off:0 Def:1),NaN,11:39,1,4.0,...,21.0,21.0,21.0,21.0,21.0,False,False,False,False,NaN
4,4,1,2,9,21801052,MISS Horford 15' Jump Shot,NaN,11:34,1,4.0,...,26.0,26.0,26.0,26.0,26.0,False,False,False,False,NaN


In [89]:
fg_getter(full_data, 2544)

0.5095890410958904

In [141]:
three_attempt_getter(NBA2019_plays, 2544)

327

In [140]:
three_attempt_getter(full_data, 2544)

327

In [41]:
file = 'test_2018-2019.csv'
time = 500
difference = 6

full_data = pd.read_csv(file)
data = full_data.copy()
    
full_data['DIFFERENCE'] = abs(full_data['HOME_SCORE'] - full_data['AWAY_SCORE'])
full_data['3PT_ATT_HOME']= pd.array(full_data['HOMEDESCRIPTION'].str.contains('3PT'), dtype="boolean")
full_data['3PT_ATT_AWAY']= pd.array(full_data['VISITORDESCRIPTION'].str.contains('3PT'), dtype="boolean")
full_data['3PT_ATT'] = ((full_data['3PT_ATT_HOME']) | (full_data['3PT_ATT_AWAY']))
full_data['3PT_MADE'] = ((full_data['3PT_ATT']) & (full_data['SHOT_MADE']))
    
data['DIFFERENCE'] = abs(data['HOME_SCORE'] - data['AWAY_SCORE'])
data['3PT_ATT_HOME']= pd.array(data['HOMEDESCRIPTION'].str.contains('3PT'), dtype="boolean")
data['3PT_ATT_AWAY']= pd.array(data['VISITORDESCRIPTION'].str.contains('3PT'), dtype="boolean")
data['3PT_ATT'] = ((data['3PT_ATT_HOME']) | (data['3PT_ATT_AWAY']))
data['3PT_MADE'] = ((data['3PT_ATT']) & (data['SHOT_MADE']))
data = data[['GAME_ID', 'PCTIMESTRING', 'PERIOD', 'DIFFERENCE','HOMEDESCRIPTION', 'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION', 'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ABBREVIATION', 'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ABBREVIATION', 'HOME_TEAM', 'AWAY_TEAM', 'FREE_THROW_PLAYER_ID', 'FREE_THROW_MADE', 'SHOT_PLAYER_ID', 'SHOT_MADE', '3PT_MADE']]
data = data.loc[data['PERIOD'] >= 4, :]
data['MINUTE'] = [100*int(i[0]) + int(i[1]) for i in data['PCTIMESTRING'].str.split(':')]
data = data.loc[data['MINUTE'] < time]
data = data.loc[abs(data['DIFFERENCE']) <= difference, :]
    
    
counts = data.groupby('PLAYER1_NAME').size().to_frame().rename(columns = {0: 'Count'})
players = data.groupby('PLAYER1_ID')[['PLAYER1_NAME']].agg(pd.unique)
players['FG%'] = [fg_getter(data, x) for x in players.index]
players['FG MADE'] = [fg_made_getter(data, x) for x in players.index]
players['FG ATTEMPTED'] = [fg_attempt_getter(data, x) for x in players.index]
players['FT%'] = [ft_getter(data, x) for x in players.index]
players['FT MADE'] = [ft_made_getter(data, x) for x in players.index]
players['FT ATTEMPTED'] = [ft_attempt_getter(data, x) for x in players.index]
players['3PT%'] = [three_getter(data, x) for x in players.index]
players['3PT MADE'] = [three_made_getter(data, x) for x in players.index]
players['3PT ATTEMPTED'] = [three_attempt_getter(data, x) for x in players.index]
    
players['TOTAL FG%'] = [fg_getter(full_data, x) for x in players.index]
players['TOTAL FG MADE'] = [fg_made_getter(full_data, x) for x in players.index]
players['TOTAL FG ATTEMPTED'] = [fg_attempt_getter(full_data, x) for x in players.index]
players['TOTAL FT%'] = [ft_getter(full_data, x) for x in players.index]
players['TOTAL FT MADE'] = [ft_made_getter(full_data, x) for x in players.index]
players['TOTAL FT ATTEMPTED'] = [ft_attempt_getter(full_data, x) for x in players.index]
players['TOTAL 3PT%'] = [three_getter(full_data, x) for x in players.index]
players['TOTAL 3PT MADE'] = [three_made_getter(full_data, x) for x in players.index]
players['TOTAL 3PT ATTEMPTED'] = [three_attempt_getter(full_data, x) for x in players.index]
    
players = players.loc[players['PLAYER1_NAME'].isin(list(counts.loc[counts['Count']>min_plays].index))]
    
players['FG DIFFERENCE'] = players['FG%'] - players['TOTAL FG%']
players['FT DIFFERENCE'] = players['FT%'] - players['TOTAL FT%']
players['3PT DIFFERENCE'] = players['3PT%'] - players['TOTAL 3PT%']
    
players['YEAR'] = [year_getter(file)[0] for x in players.index]
    
players = players[['PLAYER1_NAME', 'FG MADE', 'FG ATTEMPTED', 'FG%', 'TOTAL FG MADE', 'TOTAL FG ATTEMPTED', 'TOTAL FG%', 'FG DIFFERENCE','FT MADE', 'FT ATTEMPTED', 'FT%','TOTAL FT MADE', 'TOTAL FT ATTEMPTED', 'TOTAL FT%', 'FT DIFFERENCE', '3PT MADE', '3PT ATTEMPTED', '3PT%', 'TOTAL 3PT MADE', 'TOTAL 3PT ATTEMPTED', 'TOTAL 3PT%', '3PT DIFFERENCE', 'YEAR']]



C:\Users\Can Sarioz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (71,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: Ru

KeyError: '3PT_ATT'

In [42]:
players.head()

,PLAYER1_NAME,FG%,FG MADE,FG ATTEMPTED,FT%,FT MADE,FT ATTEMPTED
PLAYER1_ID,,,,,,,
0,NaN,NaN,0,0,NaN,0,0
520,NaN,NaN,0,0,NaN,0,0
546,NaN,NaN,0,0,NaN,0,0
548,NaN,NaN,0,0,NaN,0,0
551,NaN,NaN,0,0,NaN,0,0


In [75]:
file = 'test_2018-2019.csv'
time = 500
difference = 6

full_data = pd.read_csv(file)
data = full_data.copy()
    
full_data['DIFFERENCE'] = abs(full_data['HOME_SCORE'] - full_data['AWAY_SCORE'])
full_data['3PT_ATT_HOME']= pd.array(full_data['HOMEDESCRIPTION'].str.contains('3PT'), dtype="boolean")
full_data['3PT_ATT_AWAY']= pd.array(full_data['VISITORDESCRIPTION'].str.contains('3PT'), dtype="boolean")
full_data['3PT_ATT'] = ((full_data['3PT_ATT_HOME']) | (full_data['3PT_ATT_AWAY']))
full_data['3PT_MADE'] = ((full_data['3PT_ATT']) & (full_data['SHOT_MADE']))

C:\Users\Can Sarioz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (71,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [76]:
data['DIFFERENCE'] = abs(data['HOME_SCORE'] - data['AWAY_SCORE'])
data['3PT_ATT_HOME']= pd.array(data['HOMEDESCRIPTION'].str.contains('3PT'), dtype="boolean")
data['3PT_ATT_AWAY']= pd.array(data['VISITORDESCRIPTION'].str.contains('3PT'), dtype="boolean")
data['3PT_ATT'] = ((data['3PT_ATT_HOME']) | (data['3PT_ATT_AWAY']))
data['3PT_MADE'] = ((data['3PT_ATT']) & (data['SHOT_MADE']))
data = data[['GAME_ID', 'PCTIMESTRING', 'PERIOD', 'DIFFERENCE','HOMEDESCRIPTION', 'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION', 'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ABBREVIATION', 'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ABBREVIATION', 'HOME_TEAM', 'AWAY_TEAM', 'FREE_THROW_PLAYER_ID', 'FREE_THROW_MADE', 'SHOT_PLAYER_ID', 'SHOT_MADE', '3PT_MADE']]
data = data.loc[data['PERIOD'] >= 4, :]
data['MINUTE'] = [100*int(i[0]) + int(i[1]) for i in data['PCTIMESTRING'].str.split(':')]
data = data.loc[data['MINUTE'] < time]
data = data.loc[abs(data['DIFFERENCE']) <= difference, :]

In [47]:
counts = data.groupby('PLAYER1_NAME').size().to_frame().rename(columns = {0: 'Count'})
players = data.groupby('PLAYER1_ID')[['PLAYER1_NAME']].agg(pd.unique)
players['FG%'] = [fg_getter(data, x) for x in players.index]
players['FG MADE'] = [fg_made_getter(data, x) for x in players.index]
players['FG ATTEMPTED'] = [fg_attempt_getter(data, x) for x in players.index]
players['FT%'] = [ft_getter(data, x) for x in players.index]
players['FT MADE'] = [ft_made_getter(data, x) for x in players.index]
players['FT ATTEMPTED'] = [ft_attempt_getter(data, x) for x in players.index]
players['3PT%'] = [three_getter(data, x) for x in players.index]
players['3PT MADE'] = [three_made_getter(data, x) for x in players.index]
players['3PT ATTEMPTED'] = [three_attempt_getter(data, x) for x in players.index]
    
players['TOTAL FG%'] = [fg_getter(full_data, x) for x in players.index]
players['TOTAL FG MADE'] = [fg_made_getter(full_data, x) for x in players.index]
players['TOTAL FG ATTEMPTED'] = [fg_attempt_getter(full_data, x) for x in players.index]
players['TOTAL FT%'] = [ft_getter(full_data, x) for x in players.index]
players['TOTAL FT MADE'] = [ft_made_getter(full_data, x) for x in players.index]
players['TOTAL FT ATTEMPTED'] = [ft_attempt_getter(full_data, x) for x in players.index]
players['TOTAL 3PT%'] = [three_getter(full_data, x) for x in players.index]
players['TOTAL 3PT MADE'] = [three_made_getter(full_data, x) for x in players.index]
players['TOTAL 3PT ATTEMPTED'] = [three_attempt_getter(full_data, x) for x in players.index]

<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

KeyError: '3PT_ATT'

In [83]:
counts = data.groupby('PLAYER1_NAME').size().to_frame().rename(columns = {0: 'Count'})
players = data.groupby(['PLAYER1_ID','PLAYER1_NAME'])['PLAYER1_NAME'].agg(pd.unique).to_frame().droplevel(1)
players['FG%'] = [fg_getter(data, x) for x in players.index]
players['FG MADE'] = [fg_made_getter(data, x) for x in players.index]
players['FG ATTEMPTED'] = [fg_attempt_getter(data, x) for x in players.index]
players['FT%'] = [ft_getter(data, x) for x in players.index]
players['FT MADE'] = [ft_made_getter(data, x) for x in players.index]
players['FT ATTEMPTED'] = [ft_attempt_getter(data, x) for x in players.index]
players['3PT%'] = [three_getter(data, x) for x in players.index]
players['3PT MADE'] = [three_made_getter(data, x) for x in players.index]
players['3PT ATTEMPTED'] = [three_attempt_getter(data, x) for x in players.index]
    
players['TOTAL FG%'] = [fg_getter(full_data, x) for x in players.index]
players['TOTAL FG MADE'] = [fg_made_getter(full_data, x) for x in players.index]
players['TOTAL FG ATTEMPTED'] = [fg_attempt_getter(full_data, x) for x in players.index]
players['TOTAL FT%'] = [ft_getter(full_data, x) for x in players.index]
players['TOTAL FT MADE'] = [ft_made_getter(full_data, x) for x in players.index]
players['TOTAL FT ATTEMPTED'] = [ft_attempt_getter(full_data, x) for x in players.index]
players['TOTAL 3PT%'] = [three_getter(full_data, x) for x in players.index]
players['TOTAL 3PT MADE'] = [three_made_getter(full_data, x) for x in players.index]
players['TOTAL 3PT ATTEMPTED'] = [three_attempt_getter(full_data, x) for x in players.index]

<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-25-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

KeyError: '3PT_ATT'

In [87]:
min_plays = 127
players = players.loc[players['PLAYER1_NAME'].isin(list(counts.loc[counts['Count']>min_plays].index))]
    
players['FG DIFFERENCE'] = players['FG%'] - players['TOTAL FG%']
players['FT DIFFERENCE'] = players['FT%'] - players['TOTAL FT%']
players['3PT DIFFERENCE'] = players['3PT%'] - players['TOTAL 3PT%']
    
players['YEAR'] = [year_getter(file)[0] for x in players.index]
    


KeyError: 'TOTAL FG%'

In [88]:
players['TOTAL FG%'].head()

KeyError: 'TOTAL FG%'